In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import col
from pyspark.sql.functions import col,sum,avg,max

In [0]:
#Creating the schema for the Customer csv files
cust_schema = StructType([
    StructField("C_CUSTKEY", IntegerType()),
    StructField("C_NAME", StringType()),
    StructField("C_ADDRESS", StringType()),
    StructField("C_NATIONKEY", ShortType()),
    StructField("C_PHONE", StringType()),
    StructField("C_ACCTBAL", DoubleType()),
    StructField("C_MKTSEGMENT", StringType()),
    StructField("C_COMMENT", StringType())
])


In [0]:
df_cust_sch= spark.read.format("csv").option("header",True).schema(cust_schema).load("/mnt/Gen2/Customer/csvFiles/")

In [0]:
display(df_cust_sch.limit(10))

C_CUSTKEY,C_NAME,C_ADDRESS,C_NATIONKEY,C_PHONE,C_ACCTBAL,C_MKTSEGMENT,C_COMMENT
35165,Customer#000035165,eNQSvDTld1 f7JmY,0,10-173-541-5438,4767.46,AUTOMOBILE,special excuses. furiously pending packages
30597,Customer#000030597,S9s1dDut8Q,0,10-607-243-5581,-639.62,FURNITURE,lithely ruthless packages alongside of the blithely final p
42279,Customer#000042279,ABcVdNnA3JFB7bK5,0,10-934-981-2863,2236.39,MACHINERY,the even deposits sleep blithely packages. quickly express packages should have to detec
42578,Customer#000042578,l6VNaE7iSZFtkSC5fSuLeaoWTJgx5,0,10-281-998-8028,6429.8,BUILDING,y alongside of the platelets. regular deposits sleep fluffily blithely silent pinto beans: re
37854,Customer#000037854,dL6LCTLpY9hjLTrZ7g,0,10-909-820-4270,9549.78,BUILDING,inder blithely deposits. instructions nag quickly regular packages. regular requests
40053,Customer#000040053,qh8Q6gaffF73cm73K2R,0,10-593-423-2533,209.4,MACHINERY,c pinto beans. special instructions cajole fluffy
44060,Customer#000044060,"XXYMZ4Jd4PY3WJZA5bok4u 7oknfVG,rNobaef",0,10-519-920-9801,1150.68,BUILDING,according to the furiously bold instructions. regular ideas after th
49988,Customer#000049988,"oYYSmhl,K3t AwKzm5FmeEwz5lw hLf7z9m",0,10-704-487-3360,3708.73,BUILDING,"t quickly. pending, special accounts cajole furiously quick pinto beans. fluffily"
43569,Customer#000043569,"ANmTNESWDI17e2pG7j7min2Jm,vHJ",0,10-911-273-3629,3239.41,MACHINERY,"of the slyly even deposits. unusual, even theodolites about the slyly silent accounts bo"
27443,Customer#000027443,xOgk0us699smqWP3US4ufY MhkfbwNdJvCv,0,10-492-101-8357,4304.73,MACHINERY,ly bold accounts at the carefully final ideas nag slyly final accounts. express requests are fluffily


In [0]:
df_agg = df_cust_sch.groupBy("C_MKTSEGMENT").agg(avg("C_ACCTBAL").alias("AvgAcctBal"))

In [0]:
df_agg.show()

+------------+-----------------+
C_MKTSEGMENT| AvgAcctBal|
+------------+-----------------+
 MACHINERY| 4488.92656415906|
 AUTOMOBILE|4499.423470354938|
 BUILDING| 4508.28153208148|
 HOUSEHOLD|4500.756605717316|
 FURNITURE|4480.084685998395|
+------------+-----------------+

In [0]:
df_agg.where(df_agg.C_MKTSEGMENT=="MACHINERY").where(df_agg.AvgAcctBal>4500).show()

+------------+----------+
C_MKTSEGMENT|AvgAcctBal|
+------------+----------+
+------------+----------+

In [0]:
df_cust_sch.createOrReplaceTempView("df_Customer")

In [0]:
# Note the plans are same using DF API or SQL API
sql = spark.sql("SELECT C_MKTSEGMENT, count(1) FROM df_Customer GROUP BY C_MKTSEGMENT ")

dataframe = df_cust_sch\
  .groupBy("C_MKTSEGMENT")\
  .count()

sql.explain()
dataframe.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[C_MKTSEGMENT#370], functions=[finalmerge_count(merge count#487L) AS count(1)#470L])
 +- Exchange hashpartitioning(C_MKTSEGMENT#370, 200), true, [id=#741]
 +- HashAggregate(keys=[C_MKTSEGMENT#370], functions=[partial_count(1) AS count#487L])
 +- FileScan csv [C_MKTSEGMENT#370] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[dbfs:/mnt/Gen2/Customer/csvFiles], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<C_MKTSEGMENT:string>


== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[C_MKTSEGMENT#370], functions=[finalmerge_count(merge count#492L) AS count(1)#482L])
 +- Exchange hashpartitioning(C_MKTSEGMENT#370, 200), true, [id=#787]
 +- HashAggregate(keys=[C_MKTSEGMENT#370], functions=[partial_count(1) AS count#492L])
 +- FileScan csv [C_MKTSEGMENT#370] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[dbfs:/mnt/Gen2/Customer/csvFiles], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<C_MKTSEGMENT:string>